In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
#df1['ROLE_ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
df1['target']=df1['target'].map({1:0,0:1})

In [6]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [7]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


# Combinations with brute force 


In [8]:
from itertools import combinations

cols=list(df1.columns)
cols.remove('target')
d=df1.copy()

for j in combinations(cols,2):
    d[j[0]+'_'+j[1]]=df1[j[0]]+df1[j[1]]
    
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 22 columns):
RESOURCE                          32769 non-null object
MGR_ID                            32769 non-null object
ROLE_DEPTNAME                     32769 non-null object
ROLE_FAMILY_DESC                  32769 non-null object
ROLE_CODE                         32769 non-null object
target                            32769 non-null int64
ROLLUP_12                         32769 non-null object
RESOURCE_MGR_ID                   32769 non-null object
RESOURCE_ROLE_DEPTNAME            32769 non-null object
RESOURCE_ROLE_FAMILY_DESC         32769 non-null object
RESOURCE_ROLE_CODE                32769 non-null object
RESOURCE_ROLLUP_12                32769 non-null object
MGR_ID_ROLE_DEPTNAME              32769 non-null object
MGR_ID_ROLE_FAMILY_DESC           32769 non-null object
MGR_ID_ROLE_CODE                  32769 non-null object
MGR_ID_ROLLUP_12                  32769 non-null obj

In [9]:
def AST_ecoding(train,test,columns,target,weight):
    t=train.copy()
    v=test.copy()
    gm=t[target].mean()
    for col in columns:
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        t[col+'_enc']=t[col].map(gb['smooth'])
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
    v=v.drop(columns,axis=1)
    t=t.drop(columns,axis=1)
    return t,v

In [17]:
from category_encoders.target_encoder import TargetEncoder
from category_encoders.utils import *
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,StratifiedKFold,KFold
from sklearn.metrics import *


def kfold_AS_target(train,test,columns,target,weight,folds=10):
    
    columns=list(columns)
    kf=KFold()
    t=train
    v=test
    gm=t[target].mean()
    for col in columns:
        
        for i,j in kf.split(t):
            gb=t.loc[t.index[i]].groupby(col)[target].agg(['count','mean'])
            count=gb['count']
            mean=gb['mean']
            gb['smooth']=(count*mean + weight*gm)/(count+weight)
            t.loc[t.index[j],col+'_enc']=t.loc[t.index[j],col].map(gb['smooth'])
            
        t[col+'_enc']=t[col+'_enc'].fillna(gm)
        
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
        
    t=t.drop(columns,axis=1)
    v=v.drop(columns,axis=1)

    return t,v
        

In [11]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc

In [12]:
train,test=train_test_split(d,test_size=0.3,random_state=12)
columns=list(train.columns)
columns.remove('target')
target='target'


In [13]:
import warnings
warnings.filterwarnings('ignore')
from lightgbm import LGBMClassifier

lgb=LGBMClassifier()
high=0
high2=0

for i in tqdm(range(10,200)):
    t,v=kfold_AS_target(train,test,columns,target,i)
    xtr=t.drop(target,axis=1)
    ytr=t[target]
    xts=v.drop(target,axis=1)
    yts=v[target]
    mod=lgb.fit(xtr,ytr)
    auc=aucroc(mod,xts,yts)
    if auc>high:
        high=auc
        wt=i
    elif auc>high2:
        high2=auc
        wt2=i
print('High=',high)
print('High2=',high2)

print('Weight=',wt)
print('Weight2=',wt2)

100%|██████████| 190/190 [15:32<00:00,  4.91s/it]


High= 0.8901018350131988
High2= 0.8882626200004865
Weight= 107
Weight2= 178


In [16]:
def eliminate(algo,cols,xtr,ytr,xts,yts):
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    
    high=0

    for i in tqdm(cols):
        c=cols.copy()
        c.remove(i)

        mod=algo.fit(xtr[c],ytr)
        auc_sc=aucroc(mod,xts[c],yts)
        if auc_sc>high:
            high=auc_sc
            r=i

    return high,r


def col_select(algo,step,xtr,ytr,xts,yts):
    
    
    cols=list(xtr.columns)
    p=[]
    removed=[]
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    high_current=auc
    removed_new=[]
    high_new=auc
    best=auc
   # best_rem=[]
    while high_current-high_new<step:    


        removed=removed+removed_new
        print(len(removed))
        removed_new=[]


        if high_current==auc:
            pass
        else:
            high_current=high_new
        if len(cols)>2:
            high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
            while high_new>=high_current:
                cols.remove(r)
                removed_new.append(r)
                high_current=high_new

                high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
                print(high_new)
                if high_new>best:
                    best=high_new
                    best_rem=removed+removed_new+[r]
        else:
            break

    print('high_auc=',high_current)
    print('auc_one step ahead=',high_new)
    print('removed: ',removed)
    print('len=',len(removed))
    print('best=',best)
    print('best_rem=',best_rem)

In [15]:
t,v=kfold_AS_target(train,test,columns,target,100)
xtr=t.drop(target,axis=1)
ytr=t[target]
xts=v.drop(target,axis=1)
yts=v[target]

In [16]:
col_select(lgb,0.1,xtr,ytr,xts,yts)

0


100%|██████████| 20/20 [00:12<00:00,  1.56it/s]


0.8864183370105347
1


100%|██████████| 19/19 [00:11<00:00,  1.67it/s]


0.891470833883132


...

0.8616478150199365
17


100%|██████████| 3/3 [00:01<00:00,  2.94it/s]


0.8575171371346726
18


100%|██████████| 2/2 [00:00<00:00,  3.39it/s]


0.8191372533553695
19
high_auc= 0.8191372533553695
auc_one step ahead= 0.8191372533553695
removed:  ['ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLE_FAMILY_DESC_ROLE_CODE_enc', 'MGR_ID_ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_enc', 'MGR_ID_enc', 'RESOURCE_ROLE_FAMILY_DESC_enc', 'RESOURCE_MGR_ID_enc', 'ROLLUP_12_enc', 'MGR_ID_ROLE_DEPTNAME_enc', 'ROLE_DEPTNAME_ROLE_CODE_enc', 'ROLE_CODE_ROLLUP_12_enc', 'MGR_ID_ROLLUP_12_enc', 'RESOURCE_enc', 'ROLE_FAMILY_DESC_ROLLUP_12_enc', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_enc', 'ROLE_CODE_enc', 'RESOURCE_ROLE_DEPTNAME_enc', 'RESOURCE_ROLE_CODE_enc']
len= 19
best= 0.8933923344172047
best_rem= ['ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLE_FAMILY_DESC_ROLE_CODE_enc', 'MGR_ID_ROLE_FAMILY_DESC_enc']


In [22]:
rem=['ROLE_FAMILY_DESC_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'ROLE_FAMILY_DESC_ROLE_CODE_enc', 'MGR_ID_ROLE_FAMILY_DESC_enc']
xtr=t.drop([target]+rem,axis=1)
ytr=t[target]
xts=v.drop([target]+rem,axis=1)
yts=v[target]

In [24]:
def aucroc1(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    print('AUC=',auc)
    print('Accuracy=',acc)
    print('Confusion matrix:\n',confusion_matrix(y,pred))
    print('Classification report:\n',classification_report(y,pred))

In [26]:
mod=lgb.fit(xtr,ytr)
print('Train:\n')
aucroc1(mod,xtr,ytr)
print('Test:\n')
aucroc1(mod,xts,yts)

Train:

AUC= 0.9846590050782347
Accuracy= 0.9709216147876886
Confusion matrix:
 [[21527   101]
 [  566   744]]
Classification report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     21628
           1       0.88      0.57      0.69      1310

    accuracy                           0.97     22938
   macro avg       0.93      0.78      0.84     22938
weighted avg       0.97      0.97      0.97     22938

Test:

AUC= 0.8933923344172047
Accuracy= 0.9517851693622216
Confusion matrix:
 [[9152   92]
 [ 382  205]]
Classification report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.97      9244
           1       0.69      0.35      0.46       587

    accuracy                           0.95      9831
   macro avg       0.83      0.67      0.72      9831
weighted avg       0.94      0.95      0.94      9831



In [12]:
train,test=train_test_split(d,test_size=0.3,random_state=1772)
columns=list(train.columns)
columns.remove('target')
target='target'


In [18]:
import warnings
warnings.filterwarnings('ignore')
from lightgbm import LGBMClassifier

lgb=LGBMClassifier()
high=0
high2=0

for i in tqdm(range(10,200)):
    t,v=kfold_AS_target(train,test,columns,target,i)
    xtr=t.drop(target,axis=1)
    ytr=t[target]
    xts=v.drop(target,axis=1)
    yts=v[target]
    mod=lgb.fit(xtr,ytr)
    auc=aucroc(mod,xts,yts)
    if auc>high:
        high=auc
        wt=i
    elif auc>high2:
        high2=auc
        wt2=i
print('High=',high)
print('High2=',high2)

print('Weight=',wt)
print('Weight2=',wt2)

100%|██████████| 190/190 [23:59<00:00,  7.58s/it]


High= 0.873698514570274
High2= 0.8736385285682855
Weight= 141
Weight2= 165


In [13]:
t,v=kfold_AS_target(train,test,columns,target,100)
xtr=t.drop(target,axis=1)
ytr=t[target]
xts=v.drop(target,axis=1)
yts=v[target]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [18]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier()
col_select(lgb,0.1,xtr,ytr,xts,yts)

0


100%|██████████| 20/20 [00:11<00:00,  1.70it/s]


0.8727074039974825


100%|██████████| 19/19 [00:10<00:00,  1.78it/s]


0.8697864881626345
2


...

0.8436146721544595
17


100%|██████████| 3/3 [00:00<00:00,  3.54it/s]


0.830346650098009
18


100%|██████████| 2/2 [00:00<00:00,  3.22it/s]


0.8112497326502468
19
high_auc= 0.8112497326502468
auc_one step ahead= 0.8112497326502468
removed:  ['ROLE_FAMILY_DESC_ROLE_CODE_enc', 'ROLE_FAMILY_DESC_enc', 'MGR_ID_enc', 'MGR_ID_ROLLUP_12_enc', 'ROLE_DEPTNAME_ROLE_CODE_enc', 'ROLE_CODE_ROLLUP_12_enc', 'RESOURCE_ROLE_FAMILY_DESC_enc', 'RESOURCE_ROLE_CODE_enc', 'ROLE_DEPTNAME_ROLE_FAMILY_DESC_enc', 'ROLE_CODE_enc', 'ROLE_FAMILY_DESC_ROLLUP_12_enc', 'ROLE_DEPTNAME_ROLLUP_12_enc', 'MGR_ID_ROLE_DEPTNAME_enc', 'RESOURCE_ROLLUP_12_enc', 'ROLLUP_12_enc', 'MGR_ID_ROLE_CODE_enc', 'RESOURCE_MGR_ID_enc', 'ROLE_DEPTNAME_enc', 'RESOURCE_enc']
len= 19
best= 0.8727074039974825
best_rem= ['ROLE_FAMILY_DESC_ROLE_CODE_enc', 'ROLE_FAMILY_DESC_enc']
